In [38]:
%matplotlib widget
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np

# check gpu
tf.config.list_physical_devices('GPU') 
tf.test.is_built_with_cuda()

True

## 觀察數據集

In [39]:
(train_data, train_label), (_, _) = tf.keras.datasets.mnist.load_data()
train_data.ndim, train_data.shape, train_data.dtype

(3, (60000, 28, 28), dtype('uint8'))

In [40]:
plt.imshow(train_data[0])
plt.show()
train_label[0], train_label.dtype

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(5, dtype('uint8'))

## 撰寫讀取資料class

In [41]:
class MNISTLoader():
    def __init__(self):
        
        (self.train_data, self.train_label), (self.test_data, self.test_label) = tf.keras.datasets.mnist.load_data()
        # prep data
        self.train_data = self.train_data.astype(np.float32) / 255.0
        self.test_data = self.test_data.astype(np.float32) / 255.0
        # add color channel
        self.train_data = np.expand_dims(self.train_data, axis=-1)      # shape:[60000, 28, 28] -> [60000, 28, 28, 1]
        self.test_data = np.expand_dims(self.test_data, axis=-1)        # [10000, 28, 28] -> [10000, 28, 28, 1]
        self.train_label = self.train_label.astype(np.int32)    # uint8 -> int32
        self.test_label = self.test_label.astype(np.int32)      # uint8 -> int32
        
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]

    def get_batch(self, batch_size):
        # random batch size
        index = np.random.randint(0, self.num_train_data, batch_size)
        return self.train_data[index, :], self.train_label[index]

# 方法一

## 開發MLP Model


In [42]:
class MLP(tf.keras.Model):
    
    def __init__(self):
        super().__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense1 = tf.keras.layers.Dense(units=100, activation=tf.nn.relu)
        self.dense2 = tf.keras.layers.Dense(units=20, activation=tf.nn.leaky_relu)
        self.dense3 = tf.keras.layers.Dense(units=10)

    @tf.function
    def call(self, inputs):  # [batch_size, 28, 28, 1]
        flat1 = self.flatten(inputs)  # [batch_size, 784]
        dens1 = self.dense1(flat1)  # [batch_size, 100]
        dens2 = self.dense2(dens1)  # [batch_size, 20]
        dens3 = self.dense3(dens2)  # [batch_size, 10]
        output = tf.nn.softmax(dens3)
        return output

In [43]:
num_epochs = 5
batch_size = 2000
learning_rate = 0.0015

model = MLP()
data_loader = MNISTLoader()
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

model_name = "mnist_model1"
log_dir = 'tensorboard'
summary_writer = tf.summary.create_file_writer(log_dir)
tf.summary.trace_on(profiler=True)

num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
num_batches

150

## 訓練

In [ ]:
# @tf.function
def one_batch_step(X, y, **kwargs):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        tf.print(f"{batch_index} loss {loss}", [loss])
        with summary_writer.as_default():
            tf.summary.scalar("loss", loss, step=batch_index)
    grads = tape.gradient(loss, model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))
    
for epoch_index in range(num_epochs):
    for batch_index in range(num_batches):
        X, y = data_loader.get_batch(batch_size)
        one_batch_step(X, y, batch_index=batch_index)
with summary_writer.as_default():
    tf.summary.trace_export(name="model_trace", step=0, profiler_outdir=log_dir)
    
tf.saved_model.save(model, f"saved/{model_name}")

0 loss 2.3598012924194336 [2.35980129]
1 loss 2.2228996753692627 [2.22289968]
2 loss 2.110011339187622 [2.11001134]
3 loss 2.021075487136841 [2.02107549]
4 loss 1.917817234992981 [1.91781723]
5 loss 1.8207504749298096 [1.82075047]
6 loss 1.7157223224639893 [1.71572232]
7 loss 1.6493419408798218 [1.64934194]
8 loss 1.5577243566513062 [1.55772436]
9 loss 1.4631267786026 [1.46312678]
10 loss 1.4052798748016357 [1.40527987]
11 loss 1.300802230834961 [1.30080223]
12 loss 1.2274671792984009 [1.22746718]
13 loss 1.1776010990142822 [1.1776011]
14 loss 1.11951744556427 [1.11951745]
15 loss 1.0440082550048828 [1.04400826]
16 loss 1.0039774179458618 [1.00397742]
17 loss 0.9484937191009521 [0.948493719]
18 loss 0.9291660189628601 [0.929166]
19 loss 0.8487364053726196 [0.848736405]
20 loss 0.8097838163375854 [0.809783816]
21 loss 0.7894232273101807 [0.789423227]
22 loss 0.7549102902412415 [0.75491029]
23 loss 0.7469093203544617 [0.74690932]
24 loss 0.7053620219230652 [0.705362]
25 loss 0.6722714900

In [ ]:
model = tf.saved_model.load(f"saved/{model_name}")

def test_model(model):
    sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    num_batches = int(data_loader.num_test_data // batch_size)

    for batch_index in range(num_batches):
        start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
        y_pred = model(data_loader.test_data[start_index: end_index])
        sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred)
    print("test accuracy: %f" % sparse_categorical_accuracy.result())
    
test_model(model)

# 方法二

## Keras Pipleline

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation=tf.nn.relu),
    tf.keras.layers.Dense(units=20, activation=tf.nn.leaky_relu),
    tf.keras.layers.Dense(10),
    tf.keras.layers.Softmax()
])


model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss=tf.keras.losses.sparse_categorical_crossentropy,
    metrics=[tf.keras.metrics.sparse_categorical_accuracy]
)

model.fit(data_loader.train_data, data_loader.train_label, epochs=num_epochs, batch_size=batch_size)

print(model.evaluate(data_loader.test_data, data_loader.test_label))

model_name = "mnist_model2"
tf.saved_model.save(model, f"saved/{model_name}")

In [ ]:
model = tf.saved_model.load(f"saved/{model_name}")
test_model(model)